In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Step 1: Create a dummy dataset (replace with your own data)
X, y = make_classification(n_samples=2000, n_features=20, n_classes=2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Define the model-building function for Keras Tuner
def build_model(hp):
    model = keras.Sequential()
    
    # First hidden layer
    model.add(layers.Dense(
        units=hp.Int('units_1', min_value=16, max_value=128, step=16),
        activation=hp.Choice('activation_1', values=['relu', 'tanh'])
    ))
    
    # Optional dropout
    model.add(layers.Dropout(
        rate=hp.Choice('dropout_1', values=[0.0, 0.2, 0.5])
    ))
    
    # Second hidden layer
    model.add(layers.Dense(
        units=hp.Int('units_2', min_value=16, max_value=128, step=16),
        activation=hp.Choice('activation_2', values=['relu', 'tanh'])
    ))
    
    # Output layer (binary classification → sigmoid)
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile with tunable learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Step 3: Set up the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,           # number of different models to try
    executions_per_trial=2,  # train each model multiple times for stability
    directory='tuner_dir',
    project_name='nn_tuning'
)

# Step 4: Run the search
tuner.search(X_train, y_train,
             epochs=30,
             validation_data=(X_val, y_val),
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

# Step 5: Get the best model
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparams = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best hyperparameters:")
print(best_hyperparams.values)

# Evaluate on validation data
val_loss, val_acc = best_model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {val_acc:.4f}")


Trial 10 Complete [00h 00m 20s]
val_accuracy: 0.9074999988079071

Best val_accuracy So Far: 0.9175000190734863
Total elapsed time: 00h 01m 41s


f:\aiml2\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Best hyperparameters:
{'units_1': 128, 'activation_1': 'relu', 'dropout_1': 0.2, 'units_2': 112, 'activation_2': 'tanh', 'learning_rate': 0.0005394172197737114}
Validation Accuracy: 0.9175
